In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [ ]:
import os
import zipfile
from fastai.vision.all import *
from PIL import Image
import pandas as pd
import random
import shutil
from torchvision.transforms import ToTensor


In [ ]:
train_file_path = '/kaggle/input/aerial-cactus-identification/train.zip'
image_dir = '/kaggle/working/'

with zipfile.ZipFile(train_file_path, 'r') as zip_ref:
    zip_ref.extractall(image_dir)
    
train_list = os.listdir('/kaggle/working/train')


In [ ]:
len(train_list)


In [ ]:
for i, file_name in enumerate(train_list[:10]):
    print(f"{i+1}: {file_name}")


In [ ]:
test_file_path = '/kaggle/input/aerial-cactus-identification/test.zip'

with zipfile.ZipFile(test_file_path, 'r') as zip_ref:
    zip_ref.extractall(image_dir)
    
test_list = os.listdir('/kaggle/working/test')


In [ ]:
len(test_list)


In [ ]:
for i, file_name in enumerate(test_list[:10]):
    print(f"{i+1}: {file_name}")


In [ ]:
train_image_file_path = get_image_files('/kaggle/working/train')
train_image_file_path[0]


In [ ]:
im = Image.open(train_image_file_path[0])
im


In [ ]:
test_image_file_path = get_image_files('/kaggle/working/test')
test_image_file_path[0]


In [ ]:
im2 = Image.open(test_image_file_path[0])
im2


In [ ]:
train_csv = pd.read_csv('/kaggle/input/aerial-cactus-identification/train.csv')
test_csv = pd.read_csv('/kaggle/input/aerial-cactus-identification/sample_submission.csv')


In [ ]:
train_csv.head()


In [ ]:
test_csv.head()


In [ ]:
train_csv[train_csv['has_cactus']==1]


In [ ]:
train_csv[train_csv['has_cactus']==0]


In [ ]:
train_csv[train_csv['id']=='f11eab7bc9859cc7b77821261dcd2a0a.jpg']['has_cactus']


In [ ]:
#데이터셋 클래스 커스텀하기 
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, path, csv_file, transform=None):  
        self.path = path
        self.csv_file = csv_file
        self.transform = transform      
        
    def __len__(self):
        return len(self.path)
    
    def __getitem__(self, i):
        img = Image.open(self.path[i]).convert('RGB')
        label = self.csv_file[self.csv_file['id']==self.path[i][22:]]['has_cactus']
        
        label = label.values
        
        if self.transform:
            img = self.transform(img)
            
        return img, label


In [ ]:
# train 데이터셋 이미지를 train과 valid로 나누기
train_list = os.listdir('/kaggle/working/train')
random.shuffle(train_list)

num_valid = int(len(train_list)* 0.25)
num_train = len(train_list) - num_valid


In [ ]:
test_list = os.listdir('/kaggle/working/test')


In [ ]:
train, valid = train_list[num_valid:], train_list[:num_valid]


In [ ]:
len(train), len(valid)


In [ ]:
train_path = [os.path.join('/kaggle/working/train', file_name) for file_name in train]
valid_path = [os.path.join('/kaggle/working/train', file_name) for file_name in valid]


In [ ]:
train_path[0], valid_path[0]


In [ ]:
train_path[0][22:], valid_path[0][22:]


In [ ]:
test_path = [os.path.join('/kaggle/working/test', file_name) for file_name in test_list]
test_path[0]


In [ ]:
#train, valid 데이터셋 만들기
train_ds, valid_ds = CustomDataset(train_path, train_csv, transform=ToTensor()), CustomDataset(valid_path, train_csv, transform=ToTensor())


In [ ]:
#데이터셋 확인하기
x, y = train_ds[0]


In [ ]:
x.shape, y


In [ ]:
test_ds = CustomDataset(test_path, test_csv, transform=ToTensor())


In [ ]:
z, k = test_ds[0]
z.shape, k


In [ ]:
def collate(idxs, ds):
    xb, yb = zip(*[ds[i] for i in idxs])
    return torch.stack(xb), torch.tensor([y for y in yb], dtype=torch.int64)


In [ ]:
x, y = collate([1, 2], train_ds)
x.shape, y


In [ ]:
class DataLoader:
    def __init__(self, ds, bs=64, shuffle=False, n_workers=1):
        self.ds, self.bs, self.shuffle, self.n_workers = ds, bs, shuffle, n_workers
        
    def __len__(self): return (len(self.ds)-1)//self.bs+1
    
    def __iter__(self):
        idxs = L.range(self.ds)
        if self.shuffle: idxs = idxs.shuffle()
        chunks = [idxs[n: n+self.bs] for n in range(0, len(self.ds), self.bs)]
        with ProcessPoolExecutor(self.n_workers) as ex:
            yield from ex.map(collate, chunks, ds=self.ds)


In [ ]:
n_workers = min(16, defaults.cpus)
train_dl = DataLoader(train_ds, bs=64, shuffle=True, n_workers = n_workers)
valid_dl = DataLoader(valid_ds, bs=64, shuffle=False, n_workers = n_workers)

xb, yb = first(train_dl)


In [ ]:
test_dl = DataLoader(test_ds, bs=64, shuffle=False, n_workers=n_workers)


In [ ]:
zb, kb = first(test_dl)
zb, kb


In [ ]:
xb, yb


In [ ]:
xb.shape, yb.shape


In [ ]:
import torch.nn as nn #신경망 모듈
import torch.nn.functional as F #신경망 모듈에서 자주 사용되는 함수 


In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.layer1 = nn.Sequential(nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=2),
                                   nn.ReLU(), 
                                   nn.MaxPool2d(kernel_size=2))
        self.layer2 = nn.Sequential(nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=2),
                                   nn.ReLU(),
                                   nn.MaxPool2d(kernel_size=2))
        #평균 풀링 계층
        self.avg_pool = nn.AvgPool2d(kernel_size=2)
        #전결합 계층
        self.fc = nn.Linear(in_features=32*4*4, out_features=2)
        
    #순전파 출력 정의
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.avg_pool(x)
        x = x.view(-1, 32*4*4) #평탄화
        x = self.fc(x)
        
        return x


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')


In [ ]:
device


In [ ]:
model = Model().to(device)


In [ ]:
# 손실 함수 설정
loss_fn = nn.CrossEntropyLoss()


In [ ]:
#옵티마이저 설정
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=1e-3)


In [ ]:
len(train_dl)


In [ ]:
epochs = 10

for epoch in range(epochs):
    epoch_loss = 0
    
    for images, labels in train_dl:
        images = images.to(device)
        labels = labels.to(device)
        
        #옵티마이저 내 기울기 초기화
        optimizer.zero_grad()
        
        #순전파
        pred = model(images)
        labels = labels.squeeze(1)
        
        #손실함수를 이용하여 손실값 계산
        loss = loss_fn(pred, labels)
        
        epoch_loss += loss.item() #역전파 수행
        loss.backward()
        
        #가중치 갱신
        optimizer.step()
        
    print(f'에폭 [{epoch+1}/{epochs}] - 손실값: {epoch_loss/len(train_dl):.4f}') 


In [ ]:
#평가지표는 ROC AUC
from sklearn.metrics import roc_auc_score

# 실제값과 예측 확률값을 담을 리스트 초기화
true_list = []
preds_list = []


In [ ]:
#모델을 평가 상태로 설정
model.eval()

with torch.no_grad(): #성능 검증 시에는 기울기 계산을 비활성화
    for images, labels in valid_dl:
        images = images.to(device)
        labels = labels.to(device)
        
        #순전파
        output = model(images)
        preds = torch.softmax(output.cpu(), dim=1)[:, 1] #예측 확률
        true = labels.squeeze(1).cpu() #실젯값
        
        #예측확률과 실젯값을 리스트에 추가
        preds_list.extend(preds)
        true_list.extend(true)

        
#검증 데이터 ROC AUC 점수 계산
true_np = np.array([tensor.item() for tensor in true_list])
preds_np = np.array([tensor.item() for tensor in preds_list])

# Calculate ROC AUC
roc_auc = roc_auc_score(true_np, preds_np)
print(f'검증 데이터 ROC AUC: {roc_auc:.4f}')


In [ ]:
model.eval()
preds = []

with torch.no_grad():
    for images, _ in test_dl:
        images = images.to(device)
        
        outputs = model(images)
        
        preds_part = torch.softmax(outputs.cpu(), dim=1)[:, 1].tolist()
        
        preds.extend(preds_part)


In [ ]:
test_csv['has_cactus'] = preds
test_csv.to_csv('submission.csv', index=False)
